In [3]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import numpy as np
import time
import random

In [4]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier,ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report,auc
from sklearn.grid_search import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier 
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import cohen_kappa_score
from sklearn import cross_validation, metrics
from pprint import pprint
from sklearn import tree
from sklearn.cross_validation import StratifiedKFold
import xgboost as xgb

/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/anaconda/lib/python3.6/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [5]:
start=time.time()

#  1 导入数据

In [6]:
dfnopay=pd.read_csv("DataTech_Credit_Train_Communication1.txt") #用户违约类
dfpred=pd.read_csv("DataTech_Credit_Train_User1.txt")  #题目提供的测试集，需分割训练和测试集合
dfcomm=pd.read_csv("DataTech_公共数据_基础信息1.txt",encoding='utf-8',na_values=['\\N']) #用户基本信息。由于含有中文，需要指定utf-8编码，并指定空值类型
dfcall=pd.read_csv("DataTech_公共数据_通话1.txt",encoding='utf-8',na_values=['\\N']) #用户通话类。由于含有中文，需要指定utf-8编码，并指定空值类型
#dfnet=pd.read_csv("DataTech_公共数据_上网1.txt",encoding='utf-8',na_values=['\\N'])

dfnopay.columns=["USERI_ID","Stop_Days","Stop_Cnt","date"]
dfpred.columns=["USERI_ID","RISK_Flag"]
dfcomm.columns=["USERI_ID", "AGE", "OCCUPATION_ID", "CITY_ID", "COUNTY_ID", "ONLINE_TIME", "REAL_NAME_FLAG", "USER_CREDIT_ID", "CALL_MARK", "COMM_FLAG", "CALL_COUNTS", "VPMN_CALL_COUNTS", "TOLL_COUNTS", "WJ_CALL_COUNTS", "OUT_CALL_COUNTS", "CALLFW_COUNTS", "QQW_CALL_COUNTS", "BD_CALL_COUNTS", "ROAM_COUNTS", "CALL_DURATION_M", "BILL_DURATION_M", "VPMN_CALL_DURATION_M", "WJ_CALL_DURATION_M", "OUT_CALL_DURATION_M", "CALLFW_DURATION_M", "BD_CALL_DURATION_M", "ROAM_DURATION_M", "TOLL_DURATION_M", "QQW_CALL_DURATION_M", "GPRS_VOLUME", "ARPU", "SP_FEE", "TELE_TYPE", "TELE_FAC", "SMART_SYSTEM", "FIST_USE_DATE", "NUM_OF_COMM", "date"]
dfcall.columns=["USERI_ID", "OPP_USER_NUMBER", "START_TIME", "DURATION", "CALL_TYPE", "ROAM_TYPE", "HPLMN1", "HPLMN2", "VPLMN1", "VPLMN2", "OPP_HOME_AREACODE", "TOLL_TYPE", "industry", "serv_type"]
#dfnet.columns=["USERI_ID", "TIME_FRAME", "FIRST_DOMAIN_NAME", "APP_NAME", "APP_CLASSIFY_TYPE"]

/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (1,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
dfpred.head()

,USERI_ID,RISK_Flag
0,60015357,0
1,60015359,1
2,60015362,1
3,60015365,0
4,10032318,1


In [8]:
print("=======dfnopay======")
dfnopay.info() #数据完整
print("=======dfpred======")
dfpred.info() #数据完整
print("=======dfcomm======")
dfcomm.info() #存在空值
print("=======dfcall======")
dfcall.info() #存在空值

=======dfnopay======
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28000 entries, 0 to 27999
Data columns (total 4 columns):
USERI_ID     28000 non-null int64
Stop_Days    28000 non-null int64
Stop_Cnt     28000 non-null int64
date         28000 non-null int64
dtypes: int64(4)
memory usage: 875.1 KB
=======dfpred======
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7000 entries, 0 to 6999
Data columns (total 2 columns):
USERI_ID     7000 non-null int64
RISK_Flag    7000 non-null int64
dtypes: int64(2)
memory usage: 109.5 KB
=======dfcomm======
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83091 entries, 0 to 83090
Data columns (total 38 columns):
USERI_ID                83091 non-null int64
AGE                     79089 non-null float64
OCCUPATION_ID           83091 non-null int64
CITY_ID                 83091 non-null int64
COUNTY_ID               83091 non-null object
ONLINE_TIME             83091 non-null int64
REAL_NAME_FLAG          83091 non-null int64
USER_CREDIT_ID  

In [9]:
print("======dfnopay=======")
print(dfnopay.isnull().sum())
print("======dfpred=======")
print(dfpred.isnull().sum())
print("======dfcomm=======")
print(dfcomm.isnull().sum())
print("======dfcall=======")
print(dfcall.isnull().sum())

======dfnopay=======
USERI_ID     0
Stop_Days    0
Stop_Cnt     0
date         0
dtype: int64
======dfpred=======
USERI_ID     0
RISK_Flag    0
dtype: int64
======dfcomm=======
USERI_ID                   0
AGE                     4002
OCCUPATION_ID              0
CITY_ID                    0
COUNTY_ID                  0
ONLINE_TIME                0
REAL_NAME_FLAG             0
USER_CREDIT_ID          6241
CALL_MARK                  0
COMM_FLAG                  0
CALL_COUNTS                0
VPMN_CALL_COUNTS           0
TOLL_COUNTS                0
WJ_CALL_COUNTS             0
OUT_CALL_COUNTS            0
CALLFW_COUNTS              0
QQW_CALL_COUNTS            0
BD_CALL_COUNTS             0
ROAM_COUNTS                0
CALL_DURATION_M            0
BILL_DURATION_M            0
VPMN_CALL_DURATION_M       0
WJ_CALL_DURATION_M         0
OUT_CALL_DURATION_M        0
CALLFW_DURATION_M          0
BD_CALL_DURATION_M         0
ROAM_DURATION_M            0
TOLL_DURATION_M            0
QQW_CALL_DU

#  2 数据处理

## 2.1 空值处理

In [10]:
#dfcomm=dfcomm.dropna() #直接删除存在空值的行

In [11]:
dfcomm=dfcomm.fillna(dfcomm.median()) #用中位值填充数值型空值

In [12]:
dfcomm=dfcomm.fillna("unknown") #用unknown填充字符串型空值

## 2.2 类型转换 

In [13]:
dfnopay["USERI_ID"]=dfnopay["USERI_ID"].astype("str")
dfnopay["Stop_Days"]=dfnopay["Stop_Days"].astype("int")
dfnopay["Stop_Cnt"]=dfnopay["Stop_Cnt"].astype("int")
dfnopay["date"]=dfnopay["date"].astype("str")

In [14]:
dfpred["USERI_ID"]=dfpred["USERI_ID"].astype("str")
dfpred["RISK_Flag"]=dfpred["RISK_Flag"].astype("int")

In [15]:
dfcomm["USERI_ID"]=dfcomm["USERI_ID"].astype("str")
dfcomm["date"]=dfcomm["date"].astype("str")
dfcomm["NUM_OF_COMM"]=dfcomm["NUM_OF_COMM"].astype("int")
dfcomm["OCCUPATION_ID"]=dfcomm["OCCUPATION_ID"].astype("int")
dfcomm["CITY_ID"]=dfcomm["CITY_ID"].astype("int")
dfcomm["ONLINE_TIME"]=dfcomm["ONLINE_TIME"].astype("int")
dfcomm["REAL_NAME_FLAG"]=dfcomm["REAL_NAME_FLAG"].astype("int")
dfcomm["USER_CREDIT_ID"]=dfcomm["USER_CREDIT_ID"].astype("int")
dfcomm["CALL_MARK"]=dfcomm["CALL_MARK"].astype("int")
dfcomm["FIST_USE_DATE"]=dfcomm["FIST_USE_DATE"].astype("str")

In [16]:
dfcall["USERI_ID"]=dfcall["USERI_ID"].astype("str")
dfcall["START_TIME"]=dfcall["START_TIME"].astype("str")
dfcall["ROAM_TYPE"]=dfcall["ROAM_TYPE"].astype("str")

# 3 特征提取

## 3.1 特征1：信用历史（dfcred_f）

In [17]:
#选择6月违约天数、违约次数，形成关键数据表dfnopay06
dfcred_f=dfnopay[dfnopay.date=="201706"].drop('date',1)
dfcred_f.columns=['USERI_ID', 'Stop_Days06', 'Stop_Cnt06']

In [18]:
dfcred_f.head()

,USERI_ID,Stop_Days06,Stop_Cnt06
3,60015357,1,1
7,60015359,3,2
11,60015362,1,1
15,60015365,1,1
19,10032318,1,1


## 3.2 特征2：身份特征（dfuser_f）

In [19]:
userprofile=[
            "USERI_ID", #用户ID
            "AGE", #客户年龄
#            "OCCUPATION_ID", #客户职业类型
#            "CITY_ID", #归属地市
#            "COUNTY_ID", #归属县市
#            "ONLINE_TIME", #在网时长
#            "REAL_NAME_FLAG", #是否实名认证
            "USER_CREDIT_ID", #用户星级
#            "CALL_MARK", #通话用户标识
#            "TELE_TYPE", #终端型号
#            "TELE_FAC", #终端品牌
#            "SMART_SYSTEM", #终端操作系统
            "FIST_USE_DATE", #终端首次使用时间
           ]

In [20]:
def monthdiff(date1):
    year1=int(date1[0:4])
#    year2=int(date2[0:4])
    year2=2017
    mon1=int(date1[4:6])
#    mon2=int(date2[4:6])
    mon2=7
    return (year2-year1)*12+(mon2-mon1)

In [21]:
dfuser_f=dfcomm[userprofile]
dfuser_f.insert(1,"LAST_MONTH",pd.Series(map(monthdiff,dfuser_f.FIST_USE_DATE)))
dfuser_f=dfuser_f.drop("FIST_USE_DATE",1)

In [22]:
dfuser_f.head()

,USERI_ID,LAST_MONTH,AGE,USER_CREDIT_ID
0,60041552,25,4.0,4
1,60155603,28,4.0,5
2,60124892,5,7.0,4
3,60028102,9,4.0,4
4,60031641,20,6.0,5


## 3.3 特征3：消费能力（dfcons_f）

In [23]:
consume=[
    "USERI_ID",
    "GPRS_VOLUME",
    "ARPU",
#    "SP_FEE",
    "date"
    ]

In [24]:
dfcons_f=dfcomm[consume]

In [25]:
# 提取4月、5月、6月的数据
dfcons_f=round(dfcons_f.loc[(dfcons_f.date=="201704") | (dfcons_f.date=="201705") | (dfcons_f.date=="201706"),:].groupby(['USERI_ID'],axis=0).mean().reset_index())

In [26]:
dfcons_f.head()

,USERI_ID,GPRS_VOLUME,ARPU
0,10001228,0.0,1.0
1,10001301,0.0,2.0
2,10001366,0.0,1.0
3,10001393,0.0,1.0
4,10001460,0.0,3.0


## 3.4 特征4：人脉关系（dfsocial_f）

In [27]:
socialnw=[
    "USERI_ID",
    "NUM_OF_COMM",
    "date"
]

In [28]:
dfsocial_f=dfcomm[socialnw]

In [29]:
# 提取4月、5月、6月的数据
dfsocial_f=round(dfsocial_f.loc[(dfsocial_f.date=="201704") | (dfsocial_f.date=="201705") | (dfsocial_f.date=="201706"),:].groupby(['USERI_ID'],axis=0).mean().reset_index())

## 3.5 特征5：语音通信行为（dfcall_f）

In [30]:
callbehavior=[
            "USERI_ID", #用户ID
#            "COMM_FLAG", #通信标识
#            "CALL_COUNTS", #通话次数
#            "VPMN_CALL_COUNTS", #虚拟网通话次数
            "TOLL_COUNTS", #长途通话次数
            "WJ_CALL_COUNTS", #网外通话次数
            "OUT_CALL_COUNTS", #被叫通话次数
#            "CALLFW_COUNTS", #呼转通话次数
            "QQW_CALL_COUNTS", #亲情网通话次数
#            "BD_CALL_COUNTS", #本地通话次数
            "ROAM_COUNTS", #漫游通话次数
#            "CALL_DURATION_M", #通话时长_分钟
#            "BILL_DURATION_M", #收费通话时长_分钟
#            "VPMN_CALL_DURATION_M", #vpmn通话时长_分钟
#            "WJ_CALL_DURATION_M", #网外通话时长_分钟
#            "OUT_CALL_DURATION_M", #被叫通话时长_分钟
#            "CALLFW_DURATION_M", #呼转通话时长_分钟
#            "BD_CALL_DURATION_M", #非漫游通话时长_分钟
#            "ROAM_DURATION_M", #漫游通话时长_分钟
#            "TOLL_DURATION_M", #长话通话时长_分钟
#            "QQW_CALL_DURATION_M", #亲情网通话时长_分钟
            "date" #数据日期
]

In [31]:
dfcall_f=dfcomm[callbehavior]

In [32]:
# 提取4月、5月、6月的数据
dfcall_f=round(dfcall_f.loc[(dfcall_f.date=="201704") | (dfcall_f.date=="201705") | (dfcall_f.date=="201706"),:].groupby(['USERI_ID'],axis=0).mean().reset_index())

In [33]:
dfcall_f.head()

,USERI_ID,TOLL_COUNTS,WJ_CALL_COUNTS,OUT_CALL_COUNTS,QQW_CALL_COUNTS,ROAM_COUNTS
0,10001228,1.0,4.0,9.0,7.0,0.0
1,10001301,0.0,1.0,9.0,8.0,0.0
2,10001366,0.0,1.0,5.0,5.0,0.0
3,10001393,0.0,0.0,0.0,0.0,0.0
4,10001460,9.0,9.0,9.0,7.0,9.0


## 3.6 特征6：漫游行为（dfroam_f）

In [34]:
roambehavior=[
            "USERI_ID", #用户ID
            "ROAM_TYPE"#漫游类型
]
dfroam_f=dfcall[roambehavior]
dfroam_f=pd.concat([dfroam_f,pd.get_dummies(dfroam_f.ROAM_TYPE,prefix="R")],axis=1).drop('ROAM_TYPE',1).groupby(['USERI_ID'],axis=0).count().reset_index()
#dfroam_f.columns=['USERI_ID','R0','R1','R4','R5','R6','R8']

In [35]:
dfroam_f=dfroam_f.iloc[:,0:-1]

In [36]:
dfroam_f.head()

,USERI_ID,R_0,R_1,R_4,R_5,R_6
0,10001228,50,50,50,50,50
1,10001301,37,37,37,37,37
2,10001366,48,48,48,48,48
3,10001460,880,880,880,880,880
4,10001477,55,55,55,55,55


In [37]:
#仅仅找出被叫的漫游

#roambehavior=[
#            "USERI_ID", #用户ID
#            "ROAM_TYPE",#漫游类型
#            "CALL_TYPE",#呼叫类型
#]
#dfroam_f=dfcall[roambehavior]
#dfroam_f=pd.concat([dfroam_f,pd.get_dummies(dfroam_f.ROAM_TYPE)],axis=1).drop('ROAM_TYPE',1).groupby(['USERI_ID','CALL_TYPE'],axis=0).sum().reset_index()
#dfroam_f=dfroam_f[dfroam_f.CALL_TYPE==1].drop("CALL_TYPE",1)
#dfroam_f.columns=['USERI_ID','R0','R1','R4','R5','R6','R8']

## 3.7 特征7：上网行为（dfinternet_f)

In [38]:
#netbehavior=[
#    "USERI_ID",
#    "APP_CLASSIFY_TYPE"
#    ]

In [39]:
#dfinternet_f=dfnet[netbehavior]

In [40]:
#dfinternet_f=pd.concat([dfinternet_f,pd.get_dummies(dfinternet_f.APP_CLASSIFY_TYPE)],axis=1).drop('APP_CLASSIFY_TYPE',1).groupby(['USERI_ID'],axis=0).sum().reset_index()

## 3.8 特征8：主被叫次数（dfmomt_f）

In [41]:
momtbehavior=[
            "USERI_ID", #用户ID
            "CALL_TYPE",#呼叫类型    
]

In [42]:
dfmomt_f=pd.concat([dfcall.loc[dfcall.CALL_TYPE==0,momtbehavior],dfcall.loc[dfcall.CALL_TYPE==1,momtbehavior]],axis=0)

In [43]:
dfmomt_f=pd.concat([dfmomt_f,pd.get_dummies(dfmomt_f.CALL_TYPE)],axis=1).drop('CALL_TYPE',1).groupby(['USERI_ID'],axis=0).sum().reset_index()

In [44]:
dfmomt_f.columns=['USERI_ID','MOCALL_COUNTS','MTCALL_COUNTS']

# 4 合并宽表

In [45]:
#df=dfpred.merge(dfuser_f,on="USERI_ID",how="left").merge(dfcons_f,on="USERI_ID",how="left").merge(dfcall_f,on="USERI_ID",how="left").merge(dfroam_f,on="USERI_ID",how="left").merge(dfcred_f,on="USERI_ID",how="left").merge(dfsocial_f,on="USERI_ID",how="left").merge(dfinternet_f,on="USERI_ID",how="left")
df=dfpred.merge(dfuser_f,on="USERI_ID",how="left").merge(dfcons_f,on="USERI_ID",how="left").merge(dfcall_f,on="USERI_ID",how="left").merge(dfroam_f,on="USERI_ID",how="left").merge(dfcred_f,on="USERI_ID",how="left").merge(dfsocial_f,on="USERI_ID",how="left").merge(dfmomt_f,on="USERI_ID",how="left")

In [46]:
dfuserid=df.USERI_ID

In [47]:
df=df.drop("USERI_ID",1).drop("R_5",1).drop("R_6",1)
df=df.fillna(df.median()) #用中位值填充数值型空值

In [48]:
df.columns

Index(['RISK_Flag', 'LAST_MONTH', 'AGE', 'USER_CREDIT_ID', 'GPRS_VOLUME',
       'ARPU', 'TOLL_COUNTS', 'WJ_CALL_COUNTS', 'OUT_CALL_COUNTS',
       'QQW_CALL_COUNTS', 'ROAM_COUNTS', 'R_0', 'R_1', 'R_4', 'Stop_Days06',
       'Stop_Cnt06', 'NUM_OF_COMM', 'MOCALL_COUNTS', 'MTCALL_COUNTS'],
      dtype='object')

# 5 Blending模型

In [49]:
X1_dev,X1_test,Y1_dev,Y1_test=train_test_split(df.drop('RISK_Flag',1),df.RISK_Flag,test_size=0.20)

In [50]:
#idx = np.random.permutation(df.shape[0])
#df = df.iloc[idx]

In [51]:
X=df.drop('RISK_Flag',1)
Y=df.RISK_Flag

dev_cutoff = int(len(df) * 4/5)
X_dev = X[:dev_cutoff]
Y_dev = Y[:dev_cutoff]
X_test = X[dev_cutoff:]
Y_test = Y[dev_cutoff:]

In [52]:
n_folds = 5
n_trees = 200

In [53]:
clfs = [
        RandomForestClassifier(n_estimators = n_trees, criterion = 'gini'),
        ExtraTreesClassifier(n_estimators = n_trees * 2, criterion = 'gini'),
        GradientBoostingClassifier(n_estimators = n_trees),
    ]

In [54]:
skf = list(StratifiedKFold(Y_dev, n_folds))

In [55]:
blend_train = np.zeros((X_dev.shape[0], len(clfs))) # Number of training data x Number of classifiers
blend_test = np.zeros((X_test.shape[0], len(clfs))) # Number of testing data x Number of classifiers

In [56]:
print('X_test.shape = %s' % (str(X_test.shape)))
print('blend_train.shape = %s' % (str(blend_train.shape)))
print('blend_test.shape = %s' % (str(blend_test.shape)))

X_test.shape = (5602, 18)
blend_train.shape = (22405, 3)
blend_test.shape = (5602, 3)


In [57]:
# For each classifier, we train the number of fold times (=len(skf))
for j, clf in enumerate(clfs):
    print('Training classifier [%s]' % (j))
    blend_test_j = np.zeros((X_test.shape[0], len(skf))) # Number of testing data x Number of folds , we will take the mean of the predictions later
    for i, (train_index, cv_index) in enumerate(skf):
        print('Fold [%s]' % (i))

        # This is the training and validation set
        X_train = X_dev.iloc[train_index]
        Y_train = Y_dev.iloc[train_index]
        X_cv = X_dev.iloc[cv_index]
        Y_cv = Y_dev.iloc[cv_index]

        clf.fit(X_train, Y_train)

        # This output will be the basis for our blended classifier to train against,
        # which is also the output of our classifiers
        blend_train[cv_index, j] = clf.predict(X_cv)
        blend_test_j[:, i] = clf.predict(X_test)
    # Take the mean of the predictions of the cross validation set
    blend_test[:, j] = blend_test_j.mean(1)

print('Y_dev.shape = %s' % (Y_dev.shape))

# Start blending!
bclf = LogisticRegression()
bclf.fit(blend_train, Y_dev)

# Predict now
Y_test_predict = bclf.predict(blend_test)
score = metrics.accuracy_score(Y_test, Y_test_predict)
print('Accuracy = %s' % (score))

Training classifier [0]
Fold [0]
Fold [1]
Fold [2]
Fold [3]
Fold [4]
Training classifier [1]
Fold [0]
Fold [1]
Fold [2]
Fold [3]
Fold [4]
Training classifier [2]
Fold [0]
Fold [1]
Fold [2]
Fold [3]
Fold [4]
Y_dev.shape = 22405
Accuracy = 0.839164584077


In [58]:
clf_tree=DecisionTreeClassifier()
clf_tree.fit(X_dev,Y_dev)
score_tree=cross_val_score(clf_tree,X_test,Y_test,cv=10).mean()
clf_tree.feature_importances_
y_pred_tree=clf_tree.predict(X_test)
ck_tree=cohen_kappa_score(Y_test, y_pred_tree)
print(score_tree)
print(classification_report(Y_test, y_pred_tree))
print(ck_tree)

0.75651324166
             precision    recall  f1-score   support

          0       0.79      0.77      0.78      3142
          1       0.72      0.74      0.73      2460

avg / total       0.76      0.76      0.76      5602

0.510203165778


In [59]:
clf_tree=DecisionTreeClassifier()
clf_tree.fit(X1_dev,Y1_dev)
score_tree=cross_val_score(clf_tree,X1_test,Y_test,cv=10).mean()
clf_tree.feature_importances_
y_pred_tree=clf_tree.predict(X1_test)
ck_tree=cohen_kappa_score(Y1_test, y_pred_tree)
print(score_tree)
print(classification_report(Y1_test, y_pred_tree))
print(ck_tree)

0.516242360581
             precision    recall  f1-score   support

          0       0.99      0.99      0.99      2786
          1       0.99      0.99      0.99      2816

avg / total       0.99      0.99      0.99      5602

0.977150624322
